In [7]:
# This is where the acquistion was done of the traditional data.

import requests
import pandas as pd
import numpy as np


In [8]:
url = 'https://stats.nba.com/stats/leaguegamelog'
headers= {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
           AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
          'Referer': 'https://www.nba.com/'}
payload = {
    'Counter': '1000',
    'DateFrom': '',
    'DateTo': '',
    'Direction': 'DESC',
    'LeagueID': '00',
    'PlayerOrTeam': 'T',
    'Season': '2012-13',
    'SeasonType': 'Regular Season',
    'Sorter': 'DATE'}

jsonData = requests.get(url, headers=headers, params=payload).json()


rows = jsonData['resultSets'][0]['rowSet']
columns = jsonData['resultSets'][0]['headers']

df = pd.DataFrame(rows, columns=columns)

In [9]:
#Verify all columns are included
df.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'],
      dtype='object')

In [10]:
#Checking the data for the columns we are using
df[['TEAM_NAME','MATCHUP','GAME_ID','GAME_DATE','WL','PLUS_MINUS']].sort_values(['GAME_DATE','GAME_ID'])

,TEAM_NAME,MATCHUP,GAME_ID,GAME_DATE,WL,PLUS_MINUS
2455,Washington Wizards,WAS @ CLE,0021200001,2012-10-30,L,-10
2458,Cleveland Cavaliers,CLE vs. WAS,0021200001,2012-10-30,W,10
2454,Boston Celtics,BOS @ MIA,0021200002,2012-10-30,L,-13
2457,Miami Heat,MIA vs. BOS,0021200002,2012-10-30,W,13
2456,Dallas Mavericks,DAL @ LAL,0021200003,2012-10-30,W,8
...,...,...,...,...,...,...
23,Los Angeles Lakers,LAL vs. HOU,0021201228,2013-04-17,W,4
0,Sacramento Kings,SAC vs. LAC,0021201229,2013-04-17,L,-4
12,Los Angeles Clippers,LAC @ SAC,0021201229,2013-04-17,W,4
24,Golden State Warriors,GSW @ POR,0021201230,2013-04-17,W,11


In [11]:
# If the 2nd element in the split is an @ symbol, the game is an away game
df['HOME_AWAY']= np.nan
for i in range(df.shape[0]):
    if df['MATCHUP'][i].split()[1] == '@':
        df['HOME_AWAY'][i] = 'AWAY'
    else:
        df['HOME_AWAY'][i] = 'HOME'

<ipython-input-11-c0f91187b689>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HOME_AWAY'][i] = 'HOME'
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-11-c0f91187b689>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HOME_AWAY'][i] = 'AWAY'


In [12]:
#Export file
df.to_csv(r"/Volumes/GoogleDrive/My Drive/updated_data/traditional_12_13.csv", index=False)

In [44]:
TEAMS2021 = df[['SEASON_ID','TEAM_NAME','TEAM_ABBREVIATION','TEAM_ID']].sort_values(['TEAM_NAME']).drop_duplicates(subset="TEAM_NAME") 

In [ ]:
#This was run after the data was concatanted in SQL into the all years table
# Adds the opponent column and values to our DataFrame
all_years=pd.read_csv(r"/Volumes/GoogleDrive/My Drive/updated_data/raw_data/trad_all_years.csv")
all_years['opponent']= np.nan
for i in range(all_years.shape[0]):
        all_years['opponent'][i] = all_years['matchup'][i].split()[2]
        
all_years.to_csv(r"/Volumes/GoogleDrive/My Drive/updated_data/raw_data/trad_all_years_opp.csv", index=False)